In [1]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

In [2]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [7]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['title_naver']
collection = db['naver_title_crawl']
def crawl(acode):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}

    #date = time[:8]
    pages = range(1, 300)
    total_dt = pd.DataFrame([])
    #date=='2021.08.19 00:00'

    search = "2021.08.19"


    for page in pages:
        url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(acode, page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        table = dom.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')
        
        date_list=[]
        for i in range(2, len(tt)):
            if len(tt[i].select('td > span')) > 0:
              
                date = tt[i].select('td > span')[0].text
                date_list.append(date)
        for a in range(0,len(date_list)):
            date_list[a]=date_list[a][0:10]
        if search in date_list:
            break
        for i in range(2, len(tt)):
            if len(tt[i].select('td > span')) > 0:
                title = tt[i].select('td.title > a')[0]['title']
                views = tt[i].select('td > span')[1].text
                pos = tt[i].select('td > strong')[0].text
                neg = tt[i].select('td > strong')[1].text
                put_data = {
                    'code':acode,
                    'date':date,
                    'title':title,
                    'views':views,
                    'pos':pos,
                    'neg':neg
                }
      
                db.naver_title_crawl.insert_one(put_data)

    
                
            
   # return total_dt


In [8]:
acode={"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
      "ecopro":"086520","shinsungeng":"011930","unison":"018000",
      "kakao":"035720","naver":"035420","nc_s":"036570",
      "sds":"018260","samsung_e":"005930","sk_h":"000660",
      "skc":"011790","hansol":"014680","DB":"000990",
      "lotte_shoping":"023530","shinse":"004170","samsungcnt":"028260",
      "dongsuh":"026960","kb":"105560","korea":"071050","shinhan":"055550",
      "woori":"316140","hana":"086790"}

In [9]:
#2021.9.27 18:30 기준 -  51000개 데이터 적재...
for code,v in acode.items():
    result = crawl(v)


In [10]:
'''
date_list=[]
for i in range(2, len(tt)):
    if len(tt[i].select('td > span')) > 0:
              # print(i)
        date = tt[i].select('td > span')[0].text
        date_list.append(date)
for i in range(0,20):
    date_list[i]=date_list[i][0:10]
'''


"\ndate_list=[]\nfor i in range(2, len(tt)):\n    if len(tt[i].select('td > span')) > 0:\n              # print(i)\n        date = tt[i].select('td > span')[0].text\n        date_list.append(date)\nfor i in range(0,20):\n    date_list[i]=date_list[i][0:10]\n"

In [11]:
'''
if date_list=='2021.08.19':
    break
'''

"\nif date_list=='2021.08.19':\n    break\n"